# NAPOMENA
**Za trening se koristi "new_weather.csv", a za test pri slanju resenja "new_weather_2010-2011".**

## Konstruisanje kolone vremenskih prilika

Radi predikcije popunjenosti hotela, odlucili smo da proverimo postojanje korelacije izmedju vremenskih prilika nekog dana i popunjenosti soba u hotelu.
Podaci korisceni ovde skinuti su sa sajta [Visual Crossing](https://www.visualcrossing.com/weather/weather-data-services/Rijeka/us/last15days).

Nakon sredjivanja, dobijamo novu kolonu gde je:
- Suncan dan = 0
- Kisni dan = 1
- Sneg = 0.5.

In [2]:
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
dataset = pd.read_csv("weather_2010-2011.csv")
dataset.head()

,name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,solarenergy,uvindex,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations
0,Rijeka,2010-01-01,10.8,7.5,9.2,10.8,6.0,8.8,8.0,92.6,...,2.0,1,NaN,2010-01-01T07:41:47,2010-01-01T16:29:50,0.52,"Rain, Overcast",Cloudy skies throughout the day with a chance ...,rain,"14216099999,14324099999,14120099999,1432309999..."
1,Rijeka,2010-01-02,8.6,3.5,6.2,8.5,-0.5,4.9,2.2,76.1,...,1.5,1,NaN,2010-01-02T07:41:49,2010-01-02T16:30:46,0.56,"Rain, Overcast",Cloudy skies throughout the day with a chance ...,rain,"14216099999,14324099999,14323099999,1430709999..."
2,Rijeka,2010-01-03,6.1,-1.8,2.4,6.1,-5.1,0.5,-8.6,46.6,...,7.6,4,NaN,2010-01-03T07:41:47,2010-01-03T16:31:43,0.59,"Rain, Partially cloudy",Partly cloudy throughout the day with morning ...,snow,"14216099999,14324099999,14120099999,1432309999..."
3,Rijeka,2010-01-04,3.1,-4.2,0.6,2.8,-8.1,-1.0,-2.9,78.7,...,1.6,1,NaN,2010-01-04T07:41:43,2010-01-04T16:32:43,0.63,"Snow, Rain, Partially cloudy",Partly cloudy throughout the day with rain or ...,snow,"14216099999,14324099999,14120099999,1432309999..."
4,Rijeka,2010-01-05,5.1,1.3,3.8,5.1,-0.5,2.2,2.1,89.0,...,1.7,1,NaN,2010-01-05T07:41:37,2010-01-05T16:33:44,0.66,"Snow, Rain, Overcast",Cloudy skies throughout the day with a chance ...,snow,"14216099999,14324099999,14120099999,1432309999..."


In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 638 entries, 0 to 637
Data columns (total 33 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              638 non-null    object 
 1   datetime          638 non-null    object 
 2   tempmax           638 non-null    float64
 3   tempmin           638 non-null    float64
 4   temp              638 non-null    float64
 5   feelslikemax      638 non-null    float64
 6   feelslikemin      638 non-null    float64
 7   feelslike         638 non-null    float64
 8   dew               638 non-null    float64
 9   humidity          638 non-null    float64
 10  precip            638 non-null    float64
 11  precipprob        638 non-null    int64  
 12  precipcover       638 non-null    float64
 13  preciptype        335 non-null    object 
 14  snow              638 non-null    float64
 15  snowdepth         638 non-null    float64
 16  windgust          88 non-null     float64
 1

Kolona 'perciptype' sadrzi podatak o tome da li je bila kisa, sneg ili sunce. Nju izdvajamo i kreiramo novu kolonu unutar 'new_weather.csv'.

In [5]:
import csv

def transform_preciptype(input_csv, output_csv):
    with open(input_csv, mode="r", encoding="utf-8") as infile, open(
        output_csv, mode="w", encoding="utf-8", newline=""
    ) as outfile:
        reader = csv.DictReader(infile)
        fieldnames = ["datetime", "transformed_preciptype"]  # Include datetime in the fieldnames

        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()

        for row in reader:
            preciptype = row.get("preciptype", "")
            if preciptype == "rain":
                transformed_preciptype = 1
            elif preciptype == "rain,snow":
                transformed_preciptype = 0.5
            else:
                transformed_preciptype = 0

            # Write both datetime and transformed_preciptype to the output dataset
            writer.writerow({
                "datetime": row["datetime"],
                "transformed_preciptype": transformed_preciptype
            })

input_csv_path = "weather_2010-2011.csv"
output_csv_path = "new_weather_2010-2011.csv"
transform_preciptype(input_csv_path, output_csv_path)


In [6]:
new_w = pd.read_csv("new_weather_2010-2011.csv")
new_w.head()

,datetime,transformed_preciptype
0,2010-01-01,1.0
1,2010-01-02,1.0
2,2010-01-03,1.0
3,2010-01-04,0.5
4,2010-01-05,0.5


In [7]:
new_w.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 638 entries, 0 to 637
Data columns (total 2 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   datetime                638 non-null    object 
 1   transformed_preciptype  638 non-null    float64
dtypes: float64(1), object(1)
memory usage: 10.1+ KB
